### Import Modules

In [3]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.bootstrapper as bs
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [12]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(bs)

<module 'fitter.bootstrapper' from '../fitter/bootstrapper.py'>

### Specify fit parameters

In [13]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        
        # semi-nnlo corrections
        'include_alpha_s' : True,
        'include_log' : True, 
        'include_log2' : True,
        'include_sunset' : True,
        
        # nnnlo corrections
        'include_latt_n3lo' : True,
    }, 
    'fit_type' : 'xpt-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; experimental: 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'use_bijnens_central_value' : False,
    
    'use_prior' : True,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
    'save_pickles' : False,
    'output_name' : 'fit_results',
}

In [14]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], order=p_dict['order']['fit'], F2=p_dict['F2'], 
                  include_log=p_dict['order']['include_log'], include_log2=p_dict['order']['include_log2'], 
                  include_sunset=p_dict['order']['include_sunset'], include_alpha_s=p_dict['order']['include_alpha_s'], 
                  include_latt_n3lo=p_dict['order']['include_latt_n3lo'], include_FV=(p_dict['order']['vol'] > 6), 
                  use_bijnens_central_value=p_dict['use_bijnens_central_value']
             )
    
# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, phys_point_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info(),
                              output_name=p_dict['output_name'], save_pickles=p_dict['save_pickles'])

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

/mnt/Google/Documents/School Work/Research/fk-fpi/priors/xpt-ratio.csv
Making fits...
Done!
100% complete
Time (s):  15.096856117248535
Compiling results...

Model: xpt-ratio_FKFpi_nnlo_log_logSq_sunset_alphaS_a4

Fitting to nnlo 
 with volume corrections O(0) 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.1899(51) [1.1956(19)]   (delta_su2 = -0.00408(70))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      A_a 0              -6.3 (3.5)            1            0.00000 (22)
          1               0.0 (6.5)        L_3 0            -0.0005 (28)
     A_aa 0                 56 (22)            1             0.0000 (31)
          1                  0 (54)        L_4 0           -0.00008 (34)
      A_k 0               2.4 (2.5)            1             0.0000 (50)
          1               0.0 (5.8)        L_5 0             0.0006 (11)
 

In [15]:
bootstrapper.get_error_budget(print_budget=True)

FK/Fpi = 1.1899(51) 

Partial Errors:
               FK/Fpi
---------------------
       A_a:   0.00221
      A_aa:   0.00123
       A_k:   0.00037
    A_loga:   0.00207
       A_p:   0.00032
       L_1:   0.00007
       L_2:   0.00014
       L_3:   0.00115
       L_4:   0.00012
       L_5:   0.00034
       L_6:   0.00002
       L_7:   0.00043
       L_8:   0.00011
       mpi:   0.00011
        mk:   0.00017
  lam2_chi:   0.00085
input_data:   0.00355
---------------------
     total:   0.00510



{'disc': 0.0032681232514279036,
 'chiral': 0.0013889266897505069,
 'pp_input': 0.0008695804807734333,
 'stat': 0.0035514807006431282}

In [30]:
    data_loader.save_fit_info(bootstrapper.get_fit_info(),
                              output_name='test', save_pickles=True)

Saving...
a/w0
a
L
alpha_s
mpi
mk
mss
a2DI
Fpi
FK
w0
FK/Fpi_pm
mju
mru
mjs
mrs
lam2_chi_kpi
lam2_chi_pipi
lam2_chi_kk
lam2_chi_00
lam2_chi
Done.


In [60]:
    def get_error_budget(self, print_budget=False):
        output = {}

        fit_info = self.get_fit_info()
        fk_fpi = fit_info['FK/Fpi']

        inputs = {}
        inputs.update(fit_info['prior'])
        output['disc'] = fk_fpi.partialsdev([fit_info['prior'][key]
                                             for key in ['A_a', 'A_aa', 'A_loga'] if key in fit_info['prior']])
        output['chiral'] = fk_fpi.partialsdev([fit_info['prior'][key]
                                               for key in (set(fit_info['prior']) - set(['A_a', 'A_aa', 'A_loga']))])

        phys_point = {}
        phys_point['mpi'] = fit_info['phys_point']['mpi']
        phys_point['mk'] = fit_info['phys_point']['mk']
        phys_point['lam2_chi'] = fit_info['phys_point']['lam2_chi']
        inputs.update(phys_point)
        output['pp_input'] = fk_fpi.partialsdev(phys_point)


        # Since the input data is correlated,
        # we only need to use a single variable as a proxy
        # for all of the variables; we use 'lam2_chi'
        prior = self.get_fit().prior
        input_data = {}
        input_data['input_data'] = prior['lam2_chi']
        inputs.update(input_data)
        output['stat'] = fk_fpi.partialsdev(input_data)


        if print_budget:
            print('FK/Fpi =', fit_info['FK/Fpi'], '\n')
            print(gv.fmt_errorbudget(outputs={'FK/Fpi' : fk_fpi}, inputs=inputs, percent=False, ndecimal=5, verify=True))

        return output
    
    get_error_budget(bootstrapper, print_budget=True)

FK/Fpi = 1.1899(51) 

Partial Errors:
               FK/Fpi
---------------------
       A_a:   0.00221
      A_aa:   0.00123
       A_k:   0.00037
    A_loga:   0.00207
       A_p:   0.00032
       L_1:   0.00007
       L_2:   0.00014
       L_3:   0.00115
       L_4:   0.00012
       L_5:   0.00034
       L_6:   0.00002
       L_7:   0.00043
       L_8:   0.00011
       mpi:   0.00011
        mk:   0.00017
  lam2_chi:   0.00085
input_data:   0.00355
---------------------
     total:   0.00510



{'disc': 0.0032681232514279036,
 'chiral': 0.0013889266897505069,
 'pp_input': 0.0008695804807734333,
 'stat': 0.0035514807006431282}

In [113]:
fit_info = data_loader.get_fit_info('test')['xpt-ratio_FKFpi_nnlo_log_logSq_sunset_alphaS_a4']
print(fit_info['FK/Fpi'].deriv(fit_info['prior']['L_5']))
print(fit_info['FK/Fpi'].partialsdev([fit_info['prior']['L_5']]))

0.0
0.005097019304120683


In [112]:
fit_info = bootstrapper.get_fit_info()
print(fit_info['FK/Fpi'].deriv(fit_info['prior']['L_5']))
print(fit_info['FK/Fpi'].partialsdev([fit_info['prior']['L_5']]))

0.06803861458414184
0.0003401930729207092


In [50]:
    def get_error_budget(fit_info, print_budget=False):
        output = {}
        fk_fpi = fit_info['FK/Fpi']

        inputs = {}
        #inputs.update(fit_info['prior'])
        for key in fit_info['prior']:
            inputs[key] = fit_info['prior'][key]
        output['disc'] = fk_fpi.partialsdev([fit_info['prior'][key]
                                             for key in ['A_a', 'A_aa', 'A_loga'] if key in fit_info['prior']])
        output['chiral'] = fk_fpi.partialsdev([fit_info['prior'][key]
                                               for key in (set(fit_info['prior']) - set(['A_a', 'A_aa', 'A_loga']))])

        phys_point = {}
        phys_point['mpi'] = fit_info['phys_point']['mpi']
        phys_point['mk'] = fit_info['phys_point']['mk']
        phys_point['lam2_chi'] = fit_info['phys_point']['lam2_chi']
        inputs.update(phys_point)
        output['pp_input'] = fk_fpi.partialsdev(phys_point)


        # Since the input data is correlated,
        # we only need to use a single variable as a proxy
        # for all of the variables; we use 'lam2_chi'
        #prior = self.get_fit().prior
        #input_data = {}
        #input_data['input_data'] = prior['lam2_chi']
        #inputs.update(input_data)
        #output['stat'] = fk_fpi.partialsdev(input_data)
        print(inputs)


        if print_budget:
            print('FK/Fpi =', fit_info['FK/Fpi'], '\n')
            print(gv.fmt_errorbudget(outputs={'FK/Fpi' : fk_fpi}, inputs=inputs, percent=False, ndecimal=5, verify=True))

        return output
    
    get_error_budget(bootstrapper.get_fit_info(), print_budget=True)

{'A_a': 0.0(6.5), 'A_aa': 0(54), 'A_k': 0.0(5.8), 'A_loga': 0.0(6.5), 'A_p': 0.0(5.8), 'L_1': 0.00000(24), 'L_2': 0.00000(22), 'L_3': 0.0000(31), 'L_4': 0.0000(50), 'L_5': 0.0000(50), 'L_6': 0.00000(10), 'L_7': 0.00000(34), 'L_8': 0.00000(14), 'mpi': 134.80(30), 'mk': 494.20(30), 'lam2_chi': 1.599(12)e+06}
FK/Fpi = 1.1899(51) 

Partial Errors:
              FK/Fpi
--------------------
      A_a:   0.00221
     A_aa:   0.00123
      A_k:   0.00037
   A_loga:   0.00207
      A_p:   0.00032
      L_1:   0.00007
      L_2:   0.00014
      L_3:   0.00115
      L_4:   0.00012
      L_5:   0.00034
      L_6:   0.00002
      L_7:   0.00043
      L_8:   0.00011
      mpi:   0.00011
       mk:   0.00017
 lam2_chi:   0.00085
--------------------
    total:   0.00510



<ipython-input-50-05ac27f1e2d1>:35: UserWarning: FK/Fpi partial error 0.0036559427168060426  !=  total error 0.005097019304120683
  print(gv.fmt_errorbudget(outputs={'FK/Fpi' : fk_fpi}, inputs=inputs, percent=False, ndecimal=5, verify=True))


{'disc': 0.0032681232514279036,
 'chiral': 0.0013889266897505069,
 'pp_input': 0.0008695804807734333}

In [96]:
def test(self):
    p = self.get_fit_info()['posterior']
    
    print(gv.evalcorr([p['A_a'], p['A_k'], p['A_p'], p['L_4'], p['L_5']]))
    
test(bootstrapper)

[[ 1.          0.14852927  0.16854288  0.15795649 -0.14118969]
 [ 0.14852927  1.          0.78017949  0.40069967 -0.32779122]
 [ 0.16854288  0.78017949  1.          0.09405687  0.00951327]
 [ 0.15795649  0.40069967  0.09405687  1.         -0.99295323]
 [-0.14118969 -0.32779122  0.00951327 -0.99295323  1.        ]]


In [91]:
print(gv.evalcorr(bootstrapper.get_fit_info()['posterior']))

{('A_a', 'A_a'): array([[1.]]),('A_a', 'A_aa'): array([[-0.17241389]]),('A_a', 'A_k'): array([[0.14852927]]),('A_a', 'A_loga'): array([[-0.88879273]]),('A_a', 'A_p'): array([[0.16854288]]),('A_a', 'L_1'): array([[-0.00492822]]),('A_a', 'L_2'): array([[-0.00841322]]),('A_a', 'L_3'): array([[-0.06826485]]),('A_a', 'L_4'): array([[0.15795649]]),('A_a', 'L_5'): array([[-0.14118969]]),('A_a', 'L_6'): array([[-0.00303813]]),('A_a', 'L_7'): array([[-0.02727483]]),('A_a', 'L_8'): array([[-0.00830673]]),('A_aa', 'A_a'): array([[-0.17241389]]),('A_aa', 'A_aa'): array([[1.]]),('A_aa', 'A_k'): array([[-0.49685592]]),('A_aa', 'A_loga'): array([[-0.28948502]]),('A_aa', 'A_p'): array([[-0.37722297]]),('A_aa', 'L_1'): array([[0.01402727]]),('A_aa', 'L_2'): array([[0.02398072]]),('A_aa', 'L_3'): array([[0.19645954]]),('A_aa', 'L_4'): array([[-0.414266]]),('A_aa', 'L_5'): array([[0.37461416]]),('A_aa', 'L_6'): array([[0.00915482]]),('A_aa', 'L_7'): array([[0.07557115]]),('A_aa', 'L_8'): array([[0.023192

In [ ]:
aa

In [101]:
def get_error_budget(self, print_budget=False):
    output = {}
    
    fit_info = self.get_fit_info()
    fk_fpi = fit_info['FK/Fpi']
    
    inputs = {}
    inputs.update(fit_info['prior'])
    output['disc'] = fk_fpi.partialsdev([fit_info['prior'][key] 
                                         for key in ['A_a', 'A_aa', 'A_loga'] if key in fit_info['prior']])
    output['chiral'] = fk_fpi.partialsdev([fit_info['prior'][key] 
                                           for key in (set(fit_info['prior']) - set(['A_a', 'A_aa', 'A_loga']))])
    
    phys_point = {}
    phys_point['mpi'] = fit_info['phys_point']['mpi']
    phys_point['mk'] = fit_info['phys_point']['mk']
    phys_point['lam2_chi'] = fit_info['phys_point']['lam2_chi']
    inputs.update(phys_point)
    output['pp_input'] = fk_fpi.partialsdev(phys_point)
    
    
    # Since the input data is correlated, 
    # we only need to use a single variable as a proxy
    # for all of the variables; we use 'lam2_chi'
    prior = self.get_fit().prior
    input_data = {}
    input_data['input_data'] = prior['lam2_chi']
    inputs.update(input_data)
    output['stat'] = fk_fpi.partialsdev(input_data)
    
        
    if print_budget:
        print('FK/Fpi =', fit_info['FK/Fpi'], '\n')
        print(gv.fmt_errorbudget(outputs={'FK/Fpi' : fk_fpi}, inputs=inputs, percent=False, ndecimal=5, verify=True))
    
    return output

error_budget(bootstrapper)

{'disc': 0.003268124383050167,
 'chiral': 0.0013889343219109565,
 'pp_input': 0.0008695707926884167,
 'stat': 0.003551481248093458}

In [91]:
set(bootstrapper.get_fit_info()['prior']) - set(['A_a', 'A_aa', 'A_loga'])

{'A_k', 'A_p', 'L_1', 'L_2', 'L_3', 'L_4', 'L_5', 'L_6', 'L_7', 'L_8'}

In [35]:
whatever = lambda x : np.sqrt(np.sum(a**2 for a in x))

In [81]:
whatever([0.00221, 0.00123, 0.00207])

<ipython-input-35-bc94ca1931cf>:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  whatever = lambda x : np.sqrt(np.sum(a**2 for a in x))


0.0032683176100250723

In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


bootstrapper.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined